# <center>Introduction to Quantitative Biology(BIO213-IQB)</center>
## <center>Assignment 1</center>
<h5><div style="text-align: right">Efforts By: Pritish Wadhwa (2019440)</div></h5>
<hr>

In [1]:
import numpy as np
import math as math

In [2]:
dnaSeq1 = "ATCAGAGTA" 
dnaSeq2 = "TTCAGTA"

In [3]:
rows = len(dnaSeq1) + 1
cols = len(dnaSeq2) + 1

In [4]:
matchPenalty = 2
mismatchPenalty = -1
gapPenalty = -1

In [5]:
dpMatrix = np.zeros((rows, cols), dtype = np.int)

In [6]:
visited = np.zeros((rows, cols), dtype=np.int)

In [7]:
def isValid(i, j):
    if i == 0:
        if dpMatrix[i][j-1] == 0:
            return True
        else:
            return False
    elif j == 0:
        if dpMatrix[i-1][j] == 0:
            return True
        else:
            return False
    else:
        if dpMatrix[i-1][j] == 0 and dpMatrix[i][j-1] == 0 and dpMatrix[i-1][j-1] == 0:
            return True
        else:
            return False

In [8]:
def calculateScore(seq1, seq2):
    score = 0
    for i in range(len(seq1)):
        if seq1[i] == ' ':
            continue
        elif seq1[i] == seq2[i]:
            score += matchPenalty
        elif seq1[i] != seq2[i]:
            if seq1[i] == '-' or seq2[i] == '-':
                score += gapPenalty
            else:
                score += mismatchPenalty
    return score

In [9]:
def printSequences(seq1, seq2):
    seq1 = seq1.replace("", " ")
    seq2 = seq2.replace("", " ")
    seq1 = seq1[1: -1]
    seq2 = seq2[1: -1]
    score = calculateScore(seq1, seq2)
    print(seq1)
    for i in range(len(seq1)):
        if seq1[i] == ' ' or seq1[i] != seq2[i] or seq1[i] == seq2[i] == '-':
            print(' ', end = '')
        elif seq1[i] == seq2[i]:
            print('|', end = '')
    print()
    print(seq2)
    print("The alignment score for the above sequences with given penalties is: " + str(score))
    print()

In [10]:
def printMatrix():
    for i in range(rows):
        for j in range(cols):
            print(dpMatrix[i][j], end = "\t")
        print()

In [11]:
def globalAlignment(rows, cols, seq1, seq2):
    for i in range(1, rows):
        for j in range(1, cols):
            if seq1[i-1] == seq2[j-1]:
                dpMatrix[i][j] = dpMatrix[i-1][j-1] + matchPenalty
            else:
                dpMatrix[i][j] = max(dpMatrix[i-1][j-1] + mismatchPenalty, dpMatrix[i-1][j] + gapPenalty, dpMatrix[i][j-1] + gapPenalty)

In [12]:
def localAlignment(rows, cols, seq1, seq2):
    maxVal = -math.inf
    l = []
    # x = 0
    # y = 0
    for i in range(0, rows):
        for j in range(0, cols):
            if i == 0 or j == 0:
                dpMatrix[i][j] = 0
            elif seq1[i-1] == seq2[j-1]:
                dpMatrix[i][j] = dpMatrix[i-1][j-1] + matchPenalty
            else:
                dpMatrix[i][j] = max(0, dpMatrix[i-1][j-1] + mismatchPenalty, dpMatrix[i-1][j] + gapPenalty, dpMatrix[i][j-1] + gapPenalty)
            if dpMatrix[i][j] >= maxVal:
                maxVal = dpMatrix[i][j]
                # x = i
                # y = j
    for i in range(0, rows):
        for j in range(0, cols):
            if dpMatrix[i][j] == maxVal:
                l.append((i, j))
    return l

In [13]:
def generateSequences(i, j, seq1, seq2, ans1, ans2):
    if (i == 0 and j == 0) or (dpMatrix[i][j] == 0 and isValid(i, j)):
        ans1 = ans1[::-1]
        ans2 = ans2[::-1]
        printSequences(ans1, ans2)
        return
    elif i < 0 or j < 0 or visited[i][j] == 1:
        return
    visited[i][j] = 1
    flag = 0
    if seq1[i-1] == seq2[j-1]:
        flag = 1
        generateSequences(i-1, j-1, seq1, seq2, ans1 + seq1[i-1], ans2 + seq2[j-1])
    if dpMatrix[i][j] == dpMatrix[i-1][j-1] + mismatchPenalty:
        flag = 1
        generateSequences(i-1, j-1, seq1, seq2, ans1 + seq1[i-1], ans2 + seq2[j-1])
    if dpMatrix[i][j] == dpMatrix[i-1][j] + gapPenalty:
        flag = 1
        generateSequences(i-1, j, seq1, seq2, ans1 + seq1[i-1], ans2 + '-')
    if dpMatrix[i][j] == dpMatrix[i][j-1] + gapPenalty:
        flag = 1
        generateSequences(i, j-1, seq1, seq2, ans1 + '-', ans2 + seq2[j-1])
    if flag == 0:
        generateSequences(i-1, j-1, seq1, seq2, ans1 + '-', ans2 + '-')
    visited[i][j] = 0

## Question 1)
<hr>

In [14]:
for i in range(rows):
    dpMatrix[i][0] = -1 * i
for j in range(cols):
    dpMatrix[0][j] = -1 * j 

In [15]:
globalAlignment(rows, cols, dnaSeq1, dnaSeq2)

### 1-a) 
The generated dynamic programming matrix:

In [16]:
printMatrix()

0	-1	-2	-3	-4	-5	-6	-7	
-1	-1	-2	-3	-1	-2	-3	-4	
-2	1	1	0	-1	-2	0	-1	
-3	0	0	3	2	1	0	-1	
-4	-1	-1	2	5	4	3	2	
-5	-2	-2	1	4	7	6	5	
-6	-3	-3	0	3	6	6	8	
-7	-4	-4	-1	2	5	5	7	
-8	-5	-2	-2	1	4	7	6	
-9	-6	-3	-3	0	3	6	9	


### 1-b) 
Yes, There are more than 1 optimal alingments for the given pair of DNA Sequences. This occurs because, the highest possible alignment score for the given pair of sequences is possible in more than 1 way

### 1-c)
The global allignments with their scores are:

In [17]:
generateSequences(rows - 1, cols - 1, dnaSeq1, dnaSeq2, "", "")

A T C A G A G T A
  | |     | | | |
T T C - - A G T A
The alignment score for the above sequences with given penalties is: 9

A T C A G A G T A
  | | |     | | |
T T C A - - G T A
The alignment score for the above sequences with given penalties is: 9

A T C A G A G T A
  | | | |     | |
T T C A G - - T A
The alignment score for the above sequences with given penalties is: 9



## Question 2)
<hr>

In [18]:
l = localAlignment(rows, cols, dnaSeq1, dnaSeq2)

### 2-a) 
The generated dynamic programming matrix:

In [19]:
printMatrix()

0	0	0	0	0	0	0	0	
0	0	0	0	2	1	0	2	
0	2	2	1	1	1	3	2	
0	1	1	4	3	2	2	2	
0	0	0	3	6	5	4	4	
0	0	0	2	5	8	7	6	
0	0	0	1	4	7	7	9	
0	0	0	0	3	6	6	8	
0	2	2	1	2	5	8	7	
0	1	1	1	3	4	7	10	


### 2-c)
The local allignments with their scores are:

In [20]:
for i in l:
    generateSequences(i[0], i[1], dnaSeq1, dnaSeq2, "", "")

T C A G A G T A
| |     | | | |
T C - - A G T A
The alignment score for the above sequences with given penalties is: 10

T C A G A G T A
| | |     | | |
T C A - - G T A
The alignment score for the above sequences with given penalties is: 10

T C A G A G T A
| | | |     | |
T C A G - - T A
The alignment score for the above sequences with given penalties is: 10



## Question 3)
<hr>
The changes required in the in program to perform local alignment rather then global alignment are:
<ul>
<li>For the global alignment, the entries of the 0th row and column were initialised with 0, -1, -2 and so on, whereas in the local aligment, the entries of the 0th row and column were initialised by 0.</li>
<li>For the other entries, in the dp matrix of the global alignmnet, -ve values are possible but that is not the case for the dp matrix of local alignment. There the entries can only be a whole number.</li>
<li>The above points are valid because while in the global alignment, while filling the table, only the maximum value of the adjoining cells with the added penalties are considered, whereas in the local alignment, while filling the table, the maximum of the adoining cells with added penalties with 0 is considered.</li>
<li>While reconstructing the sequence, in the global alignment, the maximum value is always at the last cell of the matrix, where as the maximum value in the local alignment can be anywhere in the matrix, not necessarily at the very end.</li>
<li>In global alignment, while only one cell can accomodate the maximum value, whereas in local alignment, the maximum value can present in multiple cells.</li>
<li>In the global alignment, the base case is reached when you are at the 0,0th cell, whereas in local alignment, the base case is reached when, you can't move anywhere from the current cell, and the current cell has the value 0 in it.</li>
</ul>

## Question 4)
<hr>

In [21]:
matchPenalty = 2
mismatchPenalty = -1
gapPenalty = -2

### Global Alignment

In [22]:
for i in range(rows):
    dpMatrix[i][0] = -1 * i
for j in range(cols):
    dpMatrix[0][j] = -1 * j 
globalAlignment(rows, cols, dnaSeq1, dnaSeq2)
generateSequences(rows - 1, cols - 1, dnaSeq1, dnaSeq2, "", "")

A T C A G A G T A
  | |     | | | |
T T C - - A G T A
The alignment score for the above sequences with given penalties is: 7

A T C A G A G T A
  | | |     | | |
T T C A - - G T A
The alignment score for the above sequences with given penalties is: 7

A T C A G A G T A
  | | | |     | |
T T C A G - - T A
The alignment score for the above sequences with given penalties is: 7



After updatind the scoring scheme, even though the generated sequence didn't change, the corresponding score did change.
This was because, as in global alignments, there has to the complete involvement of both the subsequences.
Since the difference in the 2 strings is of 2 characters, there has to be atleast 2 gaps to be inserted.

In [23]:
# localAlignment(rows, cols, dnaSeq1, dnaSeq2)

In [24]:
printMatrix()

0	-1	-2	-3	-4	-5	-6	-7	
-1	-1	-2	-3	-1	-3	-5	-4	
-2	1	1	-1	-3	-2	-1	-3	
-3	-1	0	3	1	-1	-3	-2	
-4	-3	-2	1	5	3	1	-1	
-5	-5	-4	-1	3	7	5	3	
-6	-6	-6	-3	1	5	6	7	
-7	-7	-7	-5	-1	3	4	5	
-8	-5	-5	-7	-3	1	5	3	
-9	-7	-6	-6	-5	-1	3	7	


In [25]:
printMatrix()

0	-1	-2	-3	-4	-5	-6	-7	
-1	-1	-2	-3	-1	-3	-5	-4	
-2	1	1	-1	-3	-2	-1	-3	
-3	-1	0	3	1	-1	-3	-2	
-4	-3	-2	1	5	3	1	-1	
-5	-5	-4	-1	3	7	5	3	
-6	-6	-6	-3	1	5	6	7	
-7	-7	-7	-5	-1	3	4	5	
-8	-5	-5	-7	-3	1	5	3	
-9	-7	-6	-6	-5	-1	3	7	


In [26]:
print(visited)
print(visited.shape)
printMatrix()

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
(10, 8)
0	-1	-2	-3	-4	-5	-6	-7	
-1	-1	-2	-3	-1	-3	-5	-4	
-2	1	1	-1	-3	-2	-1	-3	
-3	-1	0	3	1	-1	-3	-2	
-4	-3	-2	1	5	3	1	-1	
-5	-5	-4	-1	3	7	5	3	
-6	-6	-6	-3	1	5	6	7	
-7	-7	-7	-5	-1	3	4	5	
-8	-5	-5	-7	-3	1	5	3	
-9	-7	-6	-6	-5	-1	3	7	


In [27]:
generateSequences(rows-1, cols-1, dnaSeq1, dnaSeq2, "", "")

A T C A G A G T A
  | |     | | | |
T T C - - A G T A
The alignment score for the above sequences with given penalties is: 7

A T C A G A G T A
  | | |     | | |
T T C A - - G T A
The alignment score for the above sequences with given penalties is: 7

A T C A G A G T A
  | | | |     | |
T T C A G - - T A
The alignment score for the above sequences with given penalties is: 7



In [28]:
matchPenalty = 2
mismatchPenalty = -3
gapPenalty = -2

In [29]:
globalAlignment(rows, cols, dnaSeq1, dnaSeq2)

In [30]:
printMatrix()

0	-1	-2	-3	-4	-5	-6	-7	
-1	-3	-4	-5	-1	-3	-5	-4	
-2	1	-1	-3	-3	-4	-1	-3	
-3	-1	-2	1	-1	-3	-3	-4	
-4	-3	-4	-1	3	1	-1	-1	
-5	-5	-6	-3	1	5	3	1	
-6	-7	-8	-5	-1	3	2	5	
-7	-9	-10	-7	-3	1	0	3	
-8	-5	-7	-9	-5	-1	3	1	
-9	-7	-8	-10	-7	-3	1	5	


In [31]:
generateSequences(rows-1, cols-1, dnaSeq1, dnaSeq2, "", "")

A T C A G A G T A
  | |     | | | |
T T C - - A G T A
The alignment score for the above sequences with given penalties is: 5

A T C A G A G T A
  | | |     | | |
T T C A - - G T A
The alignment score for the above sequences with given penalties is: 5

A T C A G A G T A
  | | | |     | |
T T C A G - - T A
The alignment score for the above sequences with given penalties is: 5

